In [119]:
import cv2
import os
from matplotlib import pyplot as plt
import numpy as np

In [37]:
def emptyFolder(path):
    files = os.listdir(path)
    for f in files:
        os.remove(path + '\\' + f)

In [38]:
#define utility functions before-hand
#reshaping the photos to have minimum dimension of 400 while maintaining the aspect ratio

def shaping(size):
    if size[0]<size[1]:
        size[1] = int(400*size[1]/size[0])
        size[0] = 400
    else:
        size[0] = int(400*size[0]/size[1])
        size[1] = 400
    return size

In [39]:
#display the image in case I want to see it

def displayImage(photo):
    cv2.imshow('image', photo)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [40]:
#get rescaling factor to rescale original photo to extract a 224x224 face
def getFactors(faceList):
    factors = []
    for x in range(len(faces)):
        factors.append(224/faces[x][2])
    return factors

In [109]:
def enlargedFacesTen(x_coordinate, y_coordinate, width, height, imageShape):
    #check if width can be increased in left direction
    if x_coordinate - (0.1*width) > -1: new_x_coordinate = int(x_coordinate - (0.1*width))
    else: return x_coordinate, y_coordinate, width, height
    #check if height can be increased in left direction
    if y_coordinate - (0.1*height) > -1: new_y_coordinate = int(y_coordinate - (0.1*height))
    else: return x_coordinate, y_coordinate, width, height
    #check if width can be increased in right direction
    if x_coordinate + (1.1*width) < imageShape[1]+1: new_width = int(1.2*width)
    else: return x_coordinate, y_coordinate, width, height
    #check if height can be increased in right direction
    if y_coordinate + (1.1*height) < imageShape[0]+1: new_height = int(1.2*height)
    else: return x_coordinate, y_coordinate, width, height
    
    return new_x_coordinate, new_y_coordinate, new_width, new_height

In [108]:
def enlargedFacesTwenty(x_coordinate, y_coordinate, width, height, imageShape):
    #check if width can be increased in left direction
    if x_coordinate - (0.2*width) > -1: new_x_coordinate = int(x_coordinate - (0.2*width))
    else: return enlargedFacesTen(x_coordinate, y_coordinate, width, height, imageShape)
    #check if height can be increased in left direction
    if y_coordinate - (0.2*height) > -1: new_y_coordinate = int(y_coordinate - (0.2*height))
    else: return enlargedFacesTen(x_coordinate, y_coordinate, width, height, imageShape)
    #check if width can be increased in right direction
    if x_coordinate + (1.2*width) < imageShape[1]+1: new_width = int(1.4*width)
    else: return enlargedFacesTen(x_coordinate, y_coordinate, width, height, imageShape)
    #check if height can be increased in right direction
    if y_coordinate + (1.2*height) < imageShape[0]+1: new_height = int(1.4*height)
    else: return enlargedFacesTen(x_coordinate, y_coordinate, width, height, imageShape)
    
    return new_x_coordinate, new_y_coordinate, new_width, new_height

In [114]:
def saveScaledFaces(faceList, factors, colourImage, grayImage, imageShape, savePath):
    for x in range(len(factors)):
        newShape = [int(i*factors[x]) for i in imageShape]
        scaledGray = cv2.resize(grayImage, tuple(newShape))
        scaledColour = cv2.resize(colourImage, tuple(newShape))
        scaledFace = [int(i*factors[x]) for i in faceList[x]]
        for (ax, y, w, h) in [scaledFace]:
            #cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 1)
            cv2.imwrite(savePath + '\\' + str(ax) + str(y) + str(w) + str(h) + '_face.jpg', scaledColour[y:y+h, ax:ax+w])

In [36]:
#path of cv2 to get haar cascades

#cv2.__file__

faceCascade = cv2.CascadeClassifier('C:\\Anaconda\\lib\\site-packages\\cv2\\data\\haarcascade_frontalface_default.xml')

In [117]:
path = 'D:\\Harsh Garg\\Pictures\\Model\\F'
photos = os.listdir(path)

In [118]:
#read image and convert it to greyscale
for pics in photos:
    try:
        image = cv2.imread(path+'\\'+pics)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        #apparently image width and height are interchanged when it is read, so I manually changed it to the correct one

        shape = [np.shape(gray.T)[0],np.shape(gray.T)[1]]

        #calculate shape of the image if it has to be 400 x (400*aspect ratio)

        shape = shaping(shape)

        #resize gray and colour images

        newGray = cv2.resize(gray, tuple(shape))

        newImage = cv2.resize(image, tuple(shape))

        # Detect faces in the resized image

        faces = faceCascade.detectMultiScale(
            newGray,
            scaleFactor=1.1,
            minNeighbors=3,
            minSize=(30, 30),
            )
        if len(faces)!=0:
            enlargedFaces = []
            for x,y,w,h in faces:
                enlargedFaces.append(list(enlargedFacesTwenty(x,y,w,h,shape)))  
            newFactors = getFactors(enlargedFaces)
            saveScaledFaces(enlargedFaces, newFactors, image, gray, shape, 'D:\\Harsh Garg\\Pictures\\Detected Faces')
        else: pass
    except: pass